In [1]:
/**
 * Calculate the mean and variance of the stochastic integral:
 *
 *  E( ∫ B1(t) * B2(t) dt ) for [0, 1], where B1, B2 are two different Brownian motions.
 *
 * @author Haksun Li
 */

%use s2

// the number of discretizations
val N_T = 1000

// the number of simulations
val N_sims = 100

// the time of the first point
val t_0 = 0.0

// the time of the last point
val t_1 = 1.0

// the seed for random number generators
val seed = 1234567890L

val T: TimeGrid = EvenlySpacedGrid(t_0, t_1, N_T) // discretization
val sde: DiscreteSDE = BMSDE() // brownian motion stochastic differential equation
val x0 = 0.0
val B: RandomRealizationGenerator = RandomRealizationOfRandomProcess(sde, T, x0) // an RNG to generate random paths
B.seed(seed)

val B1B2: FiltrationFunction = object : FiltrationFunction() {
    private var Ft1: Filtration? = null
    private var Ft2: Filtration? = null
    override fun setFt(Ft: Filtration?) {
        // FT is not used in this function
        super.setFt(Ft)
        // generate a filtration for each new simulation
        Ft1 = Filtration(B.nextRealization())
        Ft2 = Filtration(B.nextRealization())
    }

    override fun evaluate(t: Int): Double {
        val B1: Double = Ft1!!.B(t)
        val B2: Double = Ft2!!.B(t)
        return B1 * B2
    }
}
    
val I: Integral = IntegralDt(B1B2)
val E = IntegralExpectation(
    I,
    t_0,
    t_1, // [0, 1]
    N_T, // discretization
    N_sims, // number of simulations
    seed // seed
)

println("the mean is %f%n".format(E.mean()))
println("the variance is %f%n".format(E.variance()))

the mean is 0.049558

the variance is 0.196298

